In [1]:
import numpy as np
import math
from pathlib import Path

In [2]:
class Qitem:
    def __init__(self, value=None, weight=None, alpha=math.sqrt(1/2), beta=math.sqrt(1/2)):
        self.alpha = alpha
        self.beta = beta
        self.value = value
        self.weight = weight
    
    def measure(self):
        return 1 if np.random.random_sample() > self.beta**2 else 0
    
    def update(self, m):
        self.alpha = m[0][0] * self.alpha + m[0][1] * self.beta
        self.beta = m[1][0] * self.alpha + m[1][1] * self.beta

In [3]:
def calc_m(delta):
    return [[math.cos(delta), -math.sin(delta)], [math.sin(delta), math.cos(delta)]]

In [4]:
def measure_individuals(Q):
    return [q.measure() for q in Q]

In [5]:
def fit_solution(Q, q):
    value = 0
    weight = 0

    for i, measure in enumerate(q):
        value += measure*(Q[i].value)
        weight += measure*(Q[i].weight)
    
    return value, weight

In [6]:
def repair_solution(Q, q, w_capacity, v_i, w_i):
    while w_i > w_capacity:
        index = np.random.randint(0, len(q)-1)
        if q[index]:
            q[index] = 0
            v_i -= Q[index].value
            w_i -= Q[index].weight
    
    return v_i, w_i

In [7]:
def get_neighborhood(Q, population):
    return [[q.measure() for q in Q] for p in range(population)]

In [8]:
def fit_and_repair(Q, s_i, w_capacity):
    v_s, w_s = fit_solution(Q, s_i)
    if w_s > w_capacity:
        v_s, w_s = repair_solution(Q, s_i, w_capacity, v_s, w_s)
    return v_s, w_s

In [15]:
def fit_and_repair_neighborhood(Q, neighborhood, w_capacity):
    solutions = []
    for i, s_i in enumerate(neighborhood):       
        solutions.append([s_i, *fit_and_repair(Q, s_i, w_capacity)])
    return solutions

In [16]:
def update_state(Q, delta, T, tabu_itt, s_i, s_c, is_best):
    for i, q in enumerate(Q):
        if T.setdefault(i, 0) == 0:
            continue
        diff = s_c[i] - s_i[i]
        if diff == 0:
            continue
        if not is_best: 
            diff *= -1
        if Q[i].alpha * Q[i].beta < 0:
            diff *= -1
        
        Q[i].update(calc_m(delta*diff))
        T[i] = tabu_itt

In [33]:
def qts(itt, delta, population, tabu_itt, file):
    Q = []
    T = dict()
    w_capacity = 0
    n_items = 0
    optimum = 0
    s_i = None
    
    best_sol = []

    with open(file) as f:
        n_items = int(f.readline().split()[1])
        w_capacity = int(f.readline().split()[1])
        optimum = int(f.readline().split()[1])
        f.readline()
        for line in f:
            _, lineV, lineW, _ = list(map(int, line.split(',')))
            Q.append(Qitem(lineV, lineW))
    
    s_i = measure_individuals(Q)

    v_i, w_i = fit_and_repair(Q, s_i, w_capacity)
    
    best_sol = [s_i, v_i, w_i]
    print('optimum', optimum)
    print('-1', best_sol[1])
    
    while itt>0:
        itt -= 1
        neighborhood = get_neighborhood(Q,population)
        solutions = fit_and_repair_neighborhood(Q, neighborhood, w_capacity)
        c_best_sol = max(solutions, key=lambda x: x[1])
        c_worst_sol = min(solutions, key=lambda x: x[1])
        
        if(c_best_sol[1]>best_sol[1] or (c_best_sol[1]==best_sol[1] and c_best_sol[2]<best_sol[2])):
            best_sol = c_best_sol[:]
            print(itt, best_sol[1])
            
        for key, value in list(T.items()):
            T[key] -= 1
            if T[key]==0:
                del T[key]
        
        update_state(Q, delta, T, tabu_itt, s_i, c_best_sol[0], True)
        s_i = measure_individuals(Q)
        
        update_state(Q, delta, T, tabu_itt, s_i, c_worst_sol[0], False)
        s_i = measure_individuals(Q)

In [41]:
teste = Path('./data/500_13.csv')
# qts(1000, 0.01*math.pi, 10, 1, teste)
qts(1000, 0.01*math.pi, 100, 2, teste)

optimum 3198
-1 2301
999 2691
998 2886
972 3120
